**Selenium EXERCISE 1:** <BR>
<ul>
<li> Open a browser 

<li>Go to tripadvisor/Restaurants

<li>Find the search text box

<li>Clear it, input the query "Sant Cugat" and send it

<li>Go to "Restarurants" and get all the links and names of top 10 restaurants in Sant cugat
<li> Store them in a list of Dictionaries {name,links}
</ul>


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re

In [2]:
# Open browser using Chrome driver
driver = webdriver.Chrome('./chromedriver')

In [3]:
# Open url
tripadvisor_url = 'https://www.tripadvisor.com/Restaurants'

driver.get(tripadvisor_url)

In [4]:
# Close cookies warning
try:
    # Wait for 'Okay' button to show. This happens only once per
    # driver session, so it might've been closed beforehand
    ok_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="_evidon-accept-button"]'))
    )
    
    ok_button.click()
except TimeoutException as e:
    print(e)

In [5]:
search_text = 'Sant Cugat'

# Get search bar
search_bar = driver.find_element_by_xpath(
    '//*[@id="component_6"]/div/div/form/input[1]'
)

# Write 'Sant Cugat' in the search bar (clear it before in case
# there was something written before)
search_bar.clear()
search_bar.send_keys(search_text)

In [6]:
# Click on first option
try:
    # Wait for first option to contain 'Sant Cugat'. Otherwise, 'Nearby is selected taken'
    WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element((By.XPATH, '//*[@id="typeahead_results"]/a[1]'), search_text)
    )
    
    # Click on 'Sant Cugat' option
    sant_cugat = driver.find_element_by_xpath('//*[@id="typeahead_results"]/a[1]')
    sant_cugat.click()
except TimeoutException as e:
    print(e)

In [7]:
# Get top restaurants
# The xpath expression gets the <a> descendant of all div's whose data-test
# attribute contains '_list_item'
restaurants = driver.find_elements_by_xpath(
    '//*[@id="component_2"]//div[contains(@data-test, "_list_item")]/span/div[1]/div[2]/div[1]/div/span/a'
)

In [8]:
# Get top 10 restaurants
top_restaurants = {restaurants[i].text: restaurants[i].get_attribute('href') for i in range(10)}

for res, link in top_restaurants.items():
    print(f'{res}\n{link}')

1. Piaceri D’ Italia Ristorante Pizzeria
https://www.tripadvisor.com/Restaurant_Review-g1080422-d19084243-Reviews-Piaceri_D_Italia_Ristorante_Pizzeria-Sant_Cugat_del_Valles_Catalonia.html
2. Restaurant Brau
https://www.tripadvisor.com/Restaurant_Review-g1080422-d10195584-Reviews-Restaurant_Brau-Sant_Cugat_del_Valles_Catalonia.html
3. Sabatic
https://www.tripadvisor.com/Restaurant_Review-g1080422-d10167691-Reviews-Sabatic-Sant_Cugat_del_Valles_Catalonia.html
4. Nemesis
https://www.tripadvisor.com/Restaurant_Review-g1080422-d11892809-Reviews-Nemesis-Sant_Cugat_del_Valles_Catalonia.html
5. Dakidaya
https://www.tripadvisor.com/Restaurant_Review-g1080422-d4546707-Reviews-Dakidaya-Sant_Cugat_del_Valles_Catalonia.html
6. 9Reinas Sant Cugat
https://www.tripadvisor.com/Restaurant_Review-g1080422-d7155184-Reviews-9Reinas_Sant_Cugat-Sant_Cugat_del_Valles_Catalonia.html
7. Kitsune Sushi Bar
https://www.tripadvisor.com/Restaurant_Review-g1080422-d5966644-Reviews-Kitsune_Sushi_Bar-Sant_Cugat_del_Val

In [9]:
# Close browser
driver.quit()

**Selenium EXERCISE 2:**

* Go to Eix Macià Cinemes web Site. ('http://www.cinemeseixmacia.com/')
* Find all the movies that are currently playing in the Cinema.
* Filter those movies that starts from at specific time (i.e. 18PM) to 1h and half later and are recomended for a specific age audience (i.e 18 y.o).
* Search in themoviedb the average ratings for those movies and select the best one. 
* Play the movie trailer.

In [10]:
# Open browser using Chrome driver
driver = webdriver.Chrome('./chromedriver')

In [11]:
# Open url
macia_url = 'http://www.cinemeseixmacia.com/'
driver.get(macia_url)

In [12]:
movies = driver.find_elements_by_xpath('//*[@id="rt-mainbody"]/div/div[3]/div[contains(@class, "peli-item")]')
print(len(movies))

11


In [13]:
# Get movie titles
movie_titles = [
    movie.find_element_by_xpath('.//div/div[2]/div/a/h4').get_attribute('textContent')
    for movie in movies
]

# Get timetables for each movie
movie_times = [
    [
        time.get_attribute('textContent').strip()
        for time in movie.find_elements_by_xpath('.//*[contains(@class, "horasessio")]/a/button')
    ]
    for movie in movies
]

# Get age ratings
movie_ages = [
    movie.find_element_by_xpath('.//*[@id="dadespeli"]/p[2]/span').get_attribute('textContent')
    for movie in movies
]

# Transform age ratings to numerical values
movie_ages = list(map(lambda age: int(age_match[0]) if (age_match := re.match(r'\d{1,2}', age)) else 0, movie_ages))

billboard = [
    {'title': title, 'times': times, 'age': age, 'ref': element}
    for title, times, age, element in zip(movie_titles, movie_times, movie_ages, movies)
]
print(billboard)

[{'title': 'ATMOS SIN TIEMPO PARA MORIR', 'times': ['18:15', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="065b7b41-634b-4289-b48f-550664291965")>}, {'title': 'DUNE', 'times': ['16:00', '19:00', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="c12dbf97-b87c-4ac4-ab99-f4b66b4f807a")>}, {'title': 'LA FAMILIA ADDAMS 2 - LA GRAN ESCAPADA', 'times': ['16:00', '18:15', '20:30', '22:45'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="b162072f-faef-4931-9126-3b164c5af817")>}, {'title': 'LA PATRULLA CANINA:LA PELICULA', 'times': ['16:00', '18:00'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="f27b5b43-0c95-4c18-9f28-27b8b148e66b")>}, {'title': 'LAS LEYES DE LA FRONTERA', 'times': ['16

In [14]:
# User data
age = 18
time = '18:00'

In [15]:
time_hours, time_minutes = tuple(map(lambda t: int(t), time.split(':')))
margin_minutes = (time_minutes + 30) % 60
margin_hours = (time_hours + 1 + (time_minutes + 30) // 60) % 24

preferred_times = [time, f'{margin_hours}:{margin_minutes if margin_minutes > 0 else "00"}']

print(preferred_times)

['18:00', '19:30']


In [16]:
# Filter data
def filter_movie(movie, age, preferred_times):
    if age < movie['age']:
        return False
    
    for time in movie['times']:
        if time >= preferred_times[0] and time <= preferred_times[1]:
            return True
    
    return False

filtered_billboard = list(filter(lambda m: filter_movie(m, age, preferred_times), billboard))
print(filtered_billboard)

[{'title': 'ATMOS SIN TIEMPO PARA MORIR', 'times': ['18:15', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="065b7b41-634b-4289-b48f-550664291965")>}, {'title': 'DUNE', 'times': ['16:00', '19:00', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="c12dbf97-b87c-4ac4-ab99-f4b66b4f807a")>}, {'title': 'LA FAMILIA ADDAMS 2 - LA GRAN ESCAPADA', 'times': ['16:00', '18:15', '20:30', '22:45'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="b162072f-faef-4931-9126-3b164c5af817")>}, {'title': 'LA PATRULLA CANINA:LA PELICULA', 'times': ['16:00', '18:00'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="f27b5b43-0c95-4c18-9f28-27b8b148e66b")>}, {'title': 'LAS LEYES DE LA FRONTERA', 'times': ['16

In [17]:
# Get ratings for each movie
import requests

In [18]:
# Read API key
with open('api_key') as f:
    api_key = f.read().strip()



In [19]:
tmdb_url = 'https://api.themoviedb.org/3/search/movie'
params = {'page': 1, 'api_key': api_key, 'region': 'ES', 'language': 'es-ES'}

for movie in filtered_billboard:
    params['query'] = movie['title']
    response = requests.get(tmdb_url, params=params)
    res_json = response.json()['results']
    movie['score'] = res_json[0]['vote_average'] if res_json else 0
    #print(res_json)

print(filtered_billboard)

[{'title': 'ATMOS SIN TIEMPO PARA MORIR', 'times': ['18:15', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="065b7b41-634b-4289-b48f-550664291965")>, 'score': 0}, {'title': 'DUNE', 'times': ['16:00', '19:00', '22:00'], 'age': 12, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="c12dbf97-b87c-4ac4-ab99-f4b66b4f807a")>, 'score': 8.1}, {'title': 'LA FAMILIA ADDAMS 2 - LA GRAN ESCAPADA', 'times': ['16:00', '18:15', '20:30', '22:45'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="b162072f-faef-4931-9126-3b164c5af817")>, 'score': 7.6}, {'title': 'LA PATRULLA CANINA:LA PELICULA', 'times': ['16:00', '18:00'], 'age': 0, 'ref': <selenium.webdriver.remote.webelement.WebElement (session="ea2e6f6e1d2ac59dfc07f8dc127ef21b", element="f27b5b43-0c95-4c18-9f28-27b8b148e66b")>, 'score': 0},

In [20]:
best_movie = max(filtered_billboard, key=lambda x: x['score'])

In [21]:
trailer_button = best_movie['ref'].find_element_by_xpath('.//div[contains(@class, "peli-botons")]/a[1]/button')
webdriver.ActionChains(driver).move_to_element(best_movie['ref']).click(trailer_button).perform()

In [22]:
driver.switch_to.frame(driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div/iframe'))

try:
    play_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="movie_player"]/div[4]/button'))
    )
        
    play_button.click()
except TimeoutException as e:
    print(e)